In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [5]:
dataset = np.load('../dataset_generation/testing_dataset_full_200keV_1000.npy')

In [6]:
all_spectra = np.add(dataset.item()['sources'], dataset.item()['backgrounds'])
all_keys = dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys)

# Train network

### Define hyperparameters

In [7]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='DNN-kfoldsfull'

In [8]:
def make_model():
    '''
    Makes a random model given some parameters.

    '''
    number_layers = choice([1, 2, 3])
    dense_nodes = 2**np.random.randint(5, 10, number_layers)
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)
    model_features = dnn_model_features(
        learining_rate=10**np.random.uniform(-4,-1),
        l2_regularization_scale=10**np.random.uniform(-2,0),
        dropout_probability=np.random.uniform(0,1),
        batch_size=2**np.random.randint(4,10),
        output_size=all_keys_binarized.shape[1],
        dense_nodes=dense_nodes,
        activation_function=choice([tf.nn.tanh,tf.nn.relu,tf.nn.sigmoid]),
        scaler=choice([make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                       make_pipeline(FunctionTransformer(np.sqrt, validate=True))]))

    model = DNN(model_features)

    return model, model_features 

### Search hyperparameters

In [ ]:
def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index) + '_dae_features', 'wb+') as f:
        pickle.dump(model_features,f)
    return None

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for network_id in range(number_hyperparameters_to_search):
    print(network_id)
    model, model_features = make_model()
    save_features(model_features,
                  model_id,
                  network_id)
    
    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        # reset model on each iteration
        model = DNN(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys_binarized[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys_binarized[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=200,
                obj_cost=model.cross_entropy,
                earlystop_cost_fn=model.f1_error,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=0.9,
                verbose=True,
                fit_batch_verbose=10,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        all_kf_errors.append(earlystop_errors_tmp)

    testing_errors.append(np.average(k_folds_errors))
    np.save('./final-models/final_test_errors_'+model_id, testing_errors)
    np.save('./final-models/final_kf_errors_'+model_id, all_kf_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

0
Epoch 10: CostFunc loss: 0.00 34.18, EarlyStop loss: 0.00 0.96
Test error at early stop: Objectives fctn: 43.93 Early stopfctn: 43.93
Epoch 10: CostFunc loss: 0.00 46.20, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 49.76 Early stopfctn: 49.76
Epoch 10: CostFunc loss: 0.00 34.37, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 63.56 Early stopfctn: 63.56
Epoch 10: CostFunc loss: 0.00 54.29, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 51.46 Early stopfctn: 51.46
Epoch 10: CostFunc loss: 0.00 37.69, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 61.51 Early stopfctn: 61.51
1
Epoch 10: CostFunc loss: 0.00 3.44, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.44 Early stopfctn: 3.44
Epoch 10: CostFunc loss: 0.00 3.45, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.44 Early stopfctn: 3.44
Epoch 10: CostFunc loss: 0.00 3.45, EarlyStop loss: 0.00 0.97
Te

Epoch 10: CostFunc loss: 0.00 3.49, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.41 Early stopfctn: 5.41
Epoch 10: CostFunc loss: 0.00 3.66, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 6.56 Early stopfctn: 6.56
Epoch 10: CostFunc loss: 0.00 3.50, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.85 Early stopfctn: 5.85
Epoch 10: CostFunc loss: 0.00 3.48, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.00 Early stopfctn: 5.00
10
Epoch 10: CostFunc loss: 0.00 5.04, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.76 Early stopfctn: 4.76
Epoch 10: CostFunc loss: 0.00 4.93, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.82 Early stopfctn: 3.82
Epoch 10: CostFunc loss: 0.00 4.04, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 6.97 Early stopfctn: 6.97
Epoch 10: CostFunc loss: 0.00 4.77, EarlyStop loss: 0.00 0.97
Test error at earl

Epoch 30: CostFunc loss: 0.00 0.69, EarlyStop loss: 0.00 0.07
Epoch 40: CostFunc loss: 0.00 0.64, EarlyStop loss: 0.00 0.07
Test error at early stop: Objectives fctn: 0.62 Early stopfctn: 0.62
Epoch 10: CostFunc loss: 0.00 1.28, EarlyStop loss: 0.00 0.15
Epoch 20: CostFunc loss: 0.00 0.89, EarlyStop loss: 0.00 0.10
Epoch 30: CostFunc loss: 0.00 0.66, EarlyStop loss: 0.00 0.06
Test error at early stop: Objectives fctn: 0.66 Early stopfctn: 0.66
Epoch 10: CostFunc loss: 0.00 1.27, EarlyStop loss: 0.00 0.16
Epoch 20: CostFunc loss: 0.00 0.85, EarlyStop loss: 0.00 0.08
Epoch 30: CostFunc loss: 0.00 0.69, EarlyStop loss: 0.00 0.07
Test error at early stop: Objectives fctn: 0.69 Early stopfctn: 0.69
Epoch 10: CostFunc loss: 0.00 1.27, EarlyStop loss: 0.00 0.14
Epoch 20: CostFunc loss: 0.00 0.88, EarlyStop loss: 0.00 0.10
Epoch 30: CostFunc loss: 0.00 0.70, EarlyStop loss: 0.00 0.07
Epoch 40: CostFunc loss: 0.00 0.67, EarlyStop loss: 0.00 0.09
Test error at early stop: Objectives fctn: 0.64 E

Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.51 Early stopfctn: 3.51
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.51 Early stopfctn: 3.51
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.49 Early stopfctn: 3.49
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.45 Early stopfctn: 3.45
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.47 Early stopfctn: 3.47
33
Epoch 10: CostFunc loss: 0.00 3.90, EarlyStop loss: 0.00 0.94
Test error at early stop: Objectives fctn: 6.44 Early stopfctn: 6.44
Epoch 10: CostFunc loss: 0.00 3.89, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 6.40 Early stopfctn: 6.40
Epoch 10: CostFunc loss: 0.00 3.85, EarlyStop loss: 0.00 0.97
Test error at earl